# Discovery Fuentes datos.gob.es

In [1]:

import pandas as pd
import requests

### Descarga del dataset

Se utilizará un dataset público proporcionado por el Instituto Nacional de Estadística de España disponible en datos.gob.es.

In [34]:
# URL del archivo CSV
URL = "https://www.ine.es/jaxiT3/files/t/csv_bdsc/48423.csv"

# Ruta específica donde se desea guardar el archivo CSV descargado
PATH = "data/tourism_spain_dataset.csv"

In [5]:
# Realizamos la solicitud HTTP para descargar el archivo
r = requests.get(URL)

# Verificamos que la solicitud fue exitosa (código de estado 200)
if r.status_code == 200:
    # Guardamos la información descargada en el archivo local
    with open(PATH, 'wb') as f:
        f.write(r.content)
    print(f"Archivo descargado exitosamente en: {PATH}")
else:
    print(f"Error al descargar el archivo. Código de estado: {r.status_code}")

Archivo descargado exitosamente en: data/tourism_spain_dataset.csv


### Carga del dataset

Cargamos nuestro dataset dentro de un dataframe de `pandas` que nos permite manipular, limpiar y visualizar fácilmente nuestros datos.

In [6]:
df = pd.read_csv(PATH, delimiter=';')

Utilizamos el método `data.info` para conocer el tipo de dato de cada columna y la cantidad de valores no nulos.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12420 entries, 0 to 12419
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Concepto turístico                12420 non-null  object
 1   Total Nacional                    12420 non-null  object
 2   Comunidades y Ciudades Autónomas  11799 non-null  object
 3   Residencia del viajero            12420 non-null  object
 4   Periodo                           12420 non-null  object
 5   Total                             11907 non-null  object
dtypes: object(6)
memory usage: 582.3+ KB


In [8]:
df.head()

,Concepto turístico,Total Nacional,Comunidades y Ciudades Autónomas,Residencia del viajero,Periodo,Total
0,Número de alojamientos turísticos ocupados,Total Nacional,NaN,Total,2023M09,1.019.000
1,Número de alojamientos turísticos ocupados,Total Nacional,NaN,Total,2023M08,1.287.000
2,Número de alojamientos turísticos ocupados,Total Nacional,NaN,Total,2023M07,1.164.000
3,Número de alojamientos turísticos ocupados,Total Nacional,NaN,Total,2023M06,775.000
4,Número de alojamientos turísticos ocupados,Total Nacional,NaN,Total,2023M05,691.000


In [9]:
df['Comunidades y Ciudades Autónomas'].unique()

array([nan, '01 Andalucía', '02 Aragón', '03 Asturias, Principado de',
       '04 Balears, Illes', '05 Canarias', '06 Cantabria',
       '07 Castilla y León', '08 Castilla - La Mancha', '09 Cataluña',
       '10 Comunitat Valenciana', '11 Extremadura', '12 Galicia',
       '13 Madrid, Comunidad de', '14 Murcia, Región de',
       '15 Navarra, Comunidad Foral de', '16 País Vasco', '17 Rioja, La',
       '18 Ceuta', '19 Melilla'], dtype=object)

In [19]:
df[['Comunidades y Ciudades Autónomas', 'Periodo', 'Concepto turístico', 'Total']].drop_duplicates().sort_values(by='Comunidades y Ciudades Autónomas')

,Comunidades y Ciudades Autónomas,Periodo,Concepto turístico,Total
207,01 Andalucía,2023M09,Número de alojamientos turísticos ocupados,NaN
8548,01 Andalucía,2018M08,Estancia media,"4,6"
8547,01 Andalucía,2018M09,Estancia media,"3,9"
8546,01 Andalucía,2018M10,Estancia media,"3,6"
8545,01 Andalucía,2018M11,Estancia media,"3,4"
...,...,...,...,...
8482,NaN,2018M05,Estancia media,"4,4"
8483,NaN,2018M04,Estancia media,"4,5"
8484,NaN,2018M03,Estancia media,"4,9"
8485,NaN,2018M02,Estancia media,"5,3"


In [33]:
df.sample(50).to_csv('test/tourism_spain_sample.csv')

In [31]:
df.query('`Comunidades y Ciudades Autónomas`== "01 Andalucía"').sample(50).to_csv('test/Andalucía_sample.csv')

Al ver la información del dataset nos damos cuenta que la columna `Total` debería ser un valor de tipo numérico en vez de tipo objeto. Esto sucede porque `pandas` interpreta los valores como string, ya que hay valores faltantes no codificados como `NaN` y, además, se está usando el punto como separador de miles y la coma como separador decimal, por lo que debemos reemplazar dichos caracteres y convertir los datos al tipo correspondiente.

Adicionalmente, reemplazaremos los valores nulos de la columna `Comunidades y Ciudades Autónomas` por la cadena de caracteres `"Total Nacional"` ya que se refiere a los datos obtenidos en el conjunto de todas las Comunidades y Ciudades Autónomas.

In [7]:
# Reemplazamos valores faltantes codificados con puntos por NaN
data = data.replace({'..':np.nan}).replace({'.':np.nan})

# Eliminamos los puntos como separadores de miles y reemplazamos las comas por puntos para separar decimales
data['Total'] = data['Total'].str.replace('.', '').str.replace(',', '.').astype(float)

# Reemplazamos los valores nulos de la columna "Comunidades y Ciudades Autónomas" por "Total Nacional"
data['Comunidades y Ciudades Autónomas'] = data['Comunidades y Ciudades Autónomas'].fillna('Total Nacional').reset_index(drop=True)

Además, convertimos el tipo de dato de la columna `Periodo` a `datetime` para mejor manipulación.

In [8]:
# Convertimos la columna "Periodo" a tipo datetime para trabajar con fechas
data['Periodo'] = pd.to_datetime(data['Periodo'], format='%YM%m')

# Extraemos el año y mes de la columna "Periodo" y creamos nuevas columnas
data['Año'] = data['Periodo'].dt.year
data['Mes'] = data['Periodo'].dt.month

#  Eliminamos la columna "Periodo" ya que no la necesitaremos más
data = data.drop('Periodo', axis=1)

In [9]:
data

,Concepto turístico,Total Nacional,Comunidades y Ciudades Autónomas,Residencia del viajero,Total,Año,Mes
0,Número de alojamientos turísticos ocupados,Total Nacional,Total Nacional,Total,1019000.0,2023,9
1,Número de alojamientos turísticos ocupados,Total Nacional,Total Nacional,Total,1287000.0,2023,8
2,Número de alojamientos turísticos ocupados,Total Nacional,Total Nacional,Total,1164000.0,2023,7
3,Número de alojamientos turísticos ocupados,Total Nacional,Total Nacional,Total,775000.0,2023,6
4,Número de alojamientos turísticos ocupados,Total Nacional,Total Nacional,Total,691000.0,2023,5
...,...,...,...,...,...,...,...
12415,Estancia media,Total Nacional,19 Melilla,Residentes en el Extranjero,NaN,2018,5
12416,Estancia media,Total Nacional,19 Melilla,Residentes en el Extranjero,NaN,2018,4
12417,Estancia media,Total Nacional,19 Melilla,Residentes en el Extranjero,NaN,2018,3
12418,Estancia media,Total Nacional,19 Melilla,Residentes en el Extranjero,NaN,2018,2


In [10]:
data.nunique()

Concepto turístico                    3
Total Nacional                        1
Comunidades y Ciudades Autónomas     20
Residencia del viajero                3
Total                               823
Año                                   6
Mes                                  12
dtype: int64

Eliminamos la columna `Total Nacional` ya que sólo tiene un valor único que no aporta utilidad al estudio.

In [11]:
data.drop('Total Nacional', axis=1, inplace=True)

Ahora, queremos rellenar los valores nulos de la columna `Total` con el promedio del respectivo mes.

In [12]:
# Definimos las columnas para agrupar
group_columns = ['Concepto turístico', 'Comunidades y Ciudades Autónomas', 'Residencia del viajero','Año','Mes']

# Agrupamos los datos en base a las columnas definidas y calculamos el promedio del "Total"
average_totals = data.groupby(group_columns)['Total'].mean()

# Fusionamos el DataFrame original con el DataFrame de promedios
data = pd.merge(data, average_totals, on=group_columns, suffixes=('', '_average'), how='left')

# Llenamos los valores nulos en la columna "Total" con el valor promedio correspondiente
data['Total'] = data['Total'].fillna(data['Total_average'])

# Eliminamos la columna de promedios ya que no es necesaria
data.drop('Total_average', axis=1,  inplace=True)

Verificamos los nulos de las columnas que nos quedaron.

In [13]:
data.isnull().sum()

Concepto turístico                     0
Comunidades y Ciudades Autónomas       0
Residencia del viajero                 0
Total                               2702
Año                                    0
Mes                                    0
dtype: int64

Eliminamos las filas con valores nulos.

In [14]:
data = data.dropna().reset_index(drop=True)

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9718 entries, 0 to 9717
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Concepto turístico                9718 non-null   object 
 1   Comunidades y Ciudades Autónomas  9718 non-null   object 
 2   Residencia del viajero            9718 non-null   object 
 3   Total                             9718 non-null   float64
 4   Año                               9718 non-null   int32  
 5   Mes                               9718 non-null   int32  
dtypes: float64(1), int32(2), object(3)
memory usage: 379.7+ KB


In [16]:
data.to_csv('../data/interim/data_cleaned.csv')